# Part 2 - Exercises - Data Transformation

### Datasets:
* Diamonds dataset from Seaborn example datasets
* Spotify dataset downloaded from [Kaggle](https://www.kaggle.com/datasets/sashankpillai/spotify-top-200-charts-20202021)

### Learning outcomes for Part 2:
* Become comfortable using binning and aggregation in encoding channels.
* Use these tools to build histograms, aggregated bar charts, and heatmaps.
* Become familiar with the `.transform_*()` syntax for applying transforms upstream of the visualization.

## Imports

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data
print("The installed Vega-Altair version is " + alt.__version__)

# Diamonds Exercise
Let's explore the diamonds dataset from the Seaborn sample datasets.

In [ ]:
diamonds_url = "../resources/datasets/diamonds.csv"
diamonds = pd.read_csv(diamonds_url)
diamonds

Here are descriptions of the dataset columns

|Variable|Description|Values|
|--- |--- |--- |
|carat|weight of the diamond|0.2-5.01|
|cut|quality of the cut|Fair, Good, Very Good, Premium, Ideal|
|color|diamond color|J (worst) to D (best)|
|clarity|measurement of how clear the diamond is|I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best)|
|depth|total depth percentage|43-79|
|table|width of top of diamond relative to widest point|43-95|
|price|price in US dollars|\$326-\$18,823|
|x|length in mm|0-10.74|
|y|width in mm|0-58.9|
|z|depth in mm|0-31.8|

### Simple scatter

Create a simple scatterplot with `price` on the y-axis and `carat` on the x-axis. Use a [circle mark](https://altair-viz.github.io/user_guide/marks/circle.html) and lower the `size` and `opacity` properties to reduce overplotting.


> **Note:** Because this dataset has more than 5,000 rows, you will get a `MaxRowsError` by default. Follow the instructions in the error message to enable the "vegafusion" data transformer to address this error, which will optimize the generated Vega specification by removing unused columns and evaluating data transformations in the Python kernel. This raises the row limit to 100k, and the limit is enforced after any data aggregations have been applied.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/diamonds1.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.data_transformers.enable("vegafusion")
alt.Chart(diamonds).mark_circle(opacity=0.2, size=10).encode(
    alt.X("carat"),
    alt.Y("price"),
)
  ```
</details>

### Simple histogram
Create a histogram of the `carat` column. This will require enabling binning on the `x` encoding, and using `count()` as the y encoding.

Click "Open in Vega Editor" from the chart's dropdown menu and notice how the dataset included in the spec is already binned an aggregated (by VegaFusion).

Use `chart.transformed_data()` to extract the binned and aggregated data as a pandas DataFrame.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/diamonds2.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.data_transformers.enable("vegafusion")
chart = alt.Chart(diamonds).mark_bar().encode(
    alt.X("carat").bin(),
    alt.Y("count()"),
)
chart.show()
chart.transformed_data()
  ```
</details>


What happens if you only use `count()` as the `y` encoding without binning `x`?

<details>
  <summary>(Show Answer)</summary>
  The bars are centered on each unique value of `carat`. The bars have a fixed width, so they may overlap each other. When binning is enabled, Vega-Altair automatically sets the bar width to match the bin intervals.
</details>

### Aggregated Bar Chart

Create a vertical bar chart showing the average `price` for each `cut` category. As before, use `chart.transformed_data()` to extract the binned and aggregated data as a pandas DataFrame.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/diamonds3.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
chart = alt.Chart(diamonds).mark_bar().encode(
    alt.X('cut'),
    alt.Y('average(price)')
)
chart.show()
chart.transformed_data()
  ```
</details>


### Heatmap

Create a heatmap with `cut` as the x encoding, `color` and the y-encoding, and `average(price)` as the heatmap color.

Then configure the scales so that the best cut and quality is in the upper right corner, and the worst cut and quality is in the bottom left corner.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/diamonds4.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(diamonds).mark_rect().encode(
    alt.X('cut').scale(domain=["Fair", "Good", "Very Good", "Premium", "Ideal"]),
    alt.Y('color').sort("descending"),
    alt.Color('average(price)')
)
  ```
</details>

## Filtered Heatmap
Repeat the Heatmap from the previous example, but this time use a filter transform to filter to diamonds larger than 1.5 carats. What do you notice about the price distribution of these diamonds?

For more information on the filter transform, see the [Vega-Altair documentation](https://altair-viz.github.io/user_guide/transform/filter.html).

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/diamonds5.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(diamonds).mark_rect().transform_filter("datum.carat > 1.5").encode(
    alt.X('cut').scale(domain=["Fair", "Good", "Very Good", "Premium", "Ideal"]),
    alt.Y('color').sort("descending"),
    alt.Color('average(price)')
)
  ```
</details>

## Spotify Exercise

We'll use the spotify dataset (downloaded from [Kaggle](https://www.kaggle.com/datasets/sashankpillai/spotify-top-200-charts-20202021)) to explore using time-based binning.

We'll load the dataset, convert the "Release Date" column to a pandas `datetime`, and filter it to includes songs released in 2018 and later.

In [ ]:
import pandas as pd
spotify_url = "../resources/datasets/spotify.csv"
df = pd.read_csv(spotify_url, na_values=" ")
df['Release Date'] = pd.to_datetime(df['Release Date'], format="mixed")
df = df[df['Release Date'] >= "2018"]

In [ ]:
df.dtypes

### Time-binned Histogram (Yearly)

Create a histogram of songs releases per year.

See the Vega-Altair [TimeUnit documentaion page](https://altair-viz.github.io/user_guide/transform/timeunit.html#timeunit-within-encoding) for reference.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/spotify1.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(df, width=500).mark_bar().encode(
    alt.X("year(Release Date)"),
    alt.Y("count()")
)
  ```
</details>


### Time-binned Histogram (Monthly)

Create a histogram of songs releases per month.

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part2/spotify2.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(df, width=500).mark_bar().encode(
  alt.X("yearmonth(Release Date)"),
  alt.Y("count()")
)
  ```
</details>

What's the difference between the `month` and `yearmonth` timeunits?

<details>
  <summary>(Show Answer)</summary>
    
The `month` time unit will group together songs released in January across every year, whereas the `yearmonth` will only group together songs release in January of the same year.
    
</details>

### Keep exploring!
Check out all of the transformation types that Vega-Altair supports in [the documentation](https://altair-viz.github.io/user_guide/transform/index.html).  Pick one we haven't discussed yet and apply it to the spotify dataset. For example, use the density transform on song tempo.
